In [15]:
from transformers import AutoTokenizer
from datasets import load_dataset

model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(example):
    return tokenizer(example['text'], truncation=True, padding='max_length', max_length=128)

from transformers import AutoModelForSequenceClassification

from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np
import pandas as pd

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [19]:
sib_languages = ['ace_Arab', 'ace_Latn', 'acm_Arab', 'acq_Arab', 'aeb_Arab', 'afr_Latn', 'ajp_Arab', 'aka_Latn', 'als_Latn', 'amh_Ethi', 'apc_Arab', 'arb_Arab', 'arb_Latn', 'ars_Arab', 'ary_Arab', 'arz_Arab', 'asm_Beng', 'ast_Latn', 'awa_Deva', 'ayr_Latn', 'azb_Arab', 'azj_Latn', 'bak_Cyrl', 'bam_Latn', 'ban_Latn', 'bel_Cyrl', 'bem_Latn', 'ben_Beng', 'bho_Deva', 'bjn_Arab', 'bjn_Latn', 'bod_Tibt', 'bos_Latn', 'bug_Latn', 'bul_Cyrl', 'cat_Latn', 'ceb_Latn', 'ces_Latn', 'cjk_Latn', 'ckb_Arab', 'crh_Latn', 'cym_Latn', 'dan_Latn', 'deu_Latn', 'dik_Latn', 'dyu_Latn', 'dzo_Tibt', 'ell_Grek', 'eng_Latn', 'epo_Latn', 'est_Latn', 'eus_Latn', 'ewe_Latn', 'fao_Latn', 'fij_Latn', 'fin_Latn', 'fon_Latn', 'fra_Latn', 'fur_Latn', 'fuv_Latn', 'gaz_Latn', 'gla_Latn', 'gle_Latn', 'glg_Latn', 'grn_Latn', 'guj_Gujr', 'hat_Latn', 'hau_Latn', 'heb_Hebr', 'hin_Deva', 'hne_Deva', 'hrv_Latn', 'hun_Latn', 'hye_Armn', 'ibo_Latn', 'ilo_Latn', 'ind_Latn', 'isl_Latn', 'ita_Latn', 'jav_Latn', 'jpn_Jpan', 'kab_Latn', 'kac_Latn', 'kam_Latn', 'kan_Knda', 'kas_Arab', 'kas_Deva', 'kat_Geor', 'kaz_Cyrl', 'kbp_Latn', 'kea_Latn', 'khk_Cyrl', 'khm_Khmr', 'kik_Latn', 'kin_Latn', 'kir_Cyrl', 'kmb_Latn', 'kmr_Latn', 'knc_Arab', 'knc_Latn', 'kon_Latn', 'kor_Hang', 'lao_Laoo', 'lij_Latn', 'lim_Latn', 'lin_Latn', 'lit_Latn', 'lmo_Latn', 'ltg_Latn', 'ltz_Latn', 'lua_Latn', 'lug_Latn', 'luo_Latn', 'lus_Latn', 'lvs_Latn', 'mag_Deva', 'mai_Deva', 'mal_Mlym', 'mar_Deva', 'min_Arab', 'min_Latn', 'mkd_Cyrl', 'mlt_Latn', 'mni_Beng', 'mos_Latn', 'mri_Latn', 'mya_Mymr', 'nld_Latn', 'nno_Latn', 'nob_Latn', 'npi_Deva', 'nqo_Nkoo', 'nso_Latn', 'nus_Latn', 'nya_Latn', 'oci_Latn', 'ory_Orya', 'pag_Latn', 'pan_Guru', 'pap_Latn', 'pbt_Arab', 'pes_Arab', 'plt_Latn', 'pol_Latn', 'por_Latn', 'prs_Arab', 'quy_Latn', 'ron_Latn', 'run_Latn', 'rus_Cyrl', 'sag_Latn', 'san_Deva', 'sat_Olck', 'scn_Latn', 'shn_Mymr', 'sin_Sinh', 'slk_Latn', 'slv_Latn', 'smo_Latn', 'sna_Latn', 'snd_Arab', 'som_Latn', 'sot_Latn', 'spa_Latn', 'srd_Latn', 'srp_Cyrl', 'ssw_Latn', 'sun_Latn', 'swe_Latn', 'swh_Latn', 'szl_Latn', 'tam_Taml', 'taq_Latn', 'taq_Tfng', 'tat_Cyrl', 'tel_Telu', 'tgk_Cyrl', 'tgl_Latn', 'tha_Thai', 'tir_Ethi', 'tpi_Latn', 'tsn_Latn', 'tso_Latn', 'tuk_Latn', 'tum_Latn', 'tur_Latn', 'twi_Latn', 'tzm_Tfng', 'uig_Arab', 'ukr_Cyrl', 'umb_Latn', 'urd_Arab', 'uzn_Latn', 'vec_Latn', 'vie_Latn', 'war_Latn', 'wol_Latn', 'xho_Latn', 'ydd_Hebr', 'yor_Latn', 'yue_Hant', 'zho_Hans', 'zho_Hant', 'zsm_Latn', 'zul_Latn']
languages_to_run = sib_languages

In [7]:

categories = ['science/technology', 'travel', 'politics', 'sports', 'health', 'entertainment', 'geography']

# Define label2id and id2label maps
label2id = {label: i for i, label in enumerate(categories)}
id2label = {i: label for label, i in label2id.items()}

def encode_labels(example):
    example["label"] = label2id[example["category"]]
    return example

In [ ]:
# This is to delete checkpoints when better checkpoint is found. Space constraints. 

from transformers import TrainerCallback, TrainingArguments, Trainer
import os
import shutil

class CleanCheckpointCallback(TrainerCallback):
    def __init__(self, output_dir):
        self.output_dir = output_dir
        self.best_checkpoint = None

    def on_save(self, args, state, control, **kwargs):
        # Keep only the newest checkpoint
        current_ckpt_dir = os.path.join(self.output_dir, f"checkpoint-{state.global_step}")
        
        # Delete the previous best if it exists and is different
        if self.best_checkpoint and self.best_checkpoint != current_ckpt_dir:
            try:
                shutil.rmtree(self.best_checkpoint)
                print(f"Deleted old checkpoint: {self.best_checkpoint}")
            except Exception as e:
                print(f"Could not delete {self.best_checkpoint}: {e}")

        # Update the latest/best checkpoint
        self.best_checkpoint = current_ckpt_dir

In [ ]:
from tqdm import tqdm
result_categories = {}
result_accuracies = {}

for language_code in languages_to_run:
    completed = []
    if language_code in completed:
        continue
    print(language_code)
    accurate = 0

    result_categories[language_code]=[]
    dataset = load_dataset("Davlan/sib200", language_code) 
    dataset = dataset.map(encode_labels) #change text to number labels. 
    size = len(dataset['test'])

    encoded_dataset = dataset.map(preprocess, batched=True)
    
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7)
    
    training_args = TrainingArguments(
        output_dir="./results",
        eval_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        num_train_epochs=5,
        weight_decay=0.01,
        save_total_limit=2,
        load_best_model_at_end=True,
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=encoded_dataset["train"],
        eval_dataset=encoded_dataset["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[CleanCheckpointCallback(training_args.output_dir)]
    )
    trainer.train()

    # Predict on test
    preds_output = trainer.predict(encoded_dataset["test"])
    preds = np.argmax(preds_output.predictions, axis=1)
    labels = preds_output.label_ids

    # Accuracy
    acc = (preds == labels).mean()
    result_accuracies[language_code] = acc    
    # Decode predictions
    pred_labels = [id2label[p] for p in preds]
    result_categories[language_code] = pred_labels
    
    df = pd.DataFrame({
    "text": dataset['test']['text'],
    "gpt_label": result_categories[language_code]
    })

    # Save to CSV
    df.to_csv("sib_results_prompt2_XLMR/{}.csv".format(language_code), index=False)
    print(acc)

    import shutil
    import os
    
    # Delete all intermediate checkpoints
    checkpoint_dir = "./results"
    for subdir in os.listdir(checkpoint_dir):
        full_path = os.path.join(checkpoint_dir, subdir)
        if subdir.startswith("checkpoint"):
            shutil.rmtree(full_path)

print(result_accuracies)


In [16]:
result_accuracies = {'ace_Arab': np.float64(0.25), 'ace_Latn': np.float64(0.25), 'acm_Arab': np.float64(0.8774509803921569), 'acq_Arab': np.float64(0.8480392156862745), 'aeb_Arab': np.float64(0.8431372549019608), 'afr_Latn': np.float64(0.8774509803921569), 'ajp_Arab': np.float64(0.8382352941176471), 'aka_Latn': np.float64(0.3382352941176471), 'als_Latn': np.float64(0.8627450980392157), 'amh_Ethi': np.float64(0.6715686274509803), 'apc_Arab': np.float64(0.8676470588235294), 'arb_Arab': np.float64(0.8284313725490197), 'arb_Latn': np.float64(0.2647058823529412), 'ars_Arab': np.float64(0.8872549019607843), 'ary_Arab': np.float64(0.8284313725490197), 'arz_Arab': np.float64(0.8333333333333334), 'asm_Beng': np.float64(0.3333333333333333), 'ast_Latn': np.float64(0.8529411764705882), 'awa_Deva': np.float64(0.8480392156862745), 'ayr_Latn': np.float64(0.25), 'azb_Arab': np.float64(0.7254901960784313), 'azj_Latn': np.float64(0.8186274509803921), 'bak_Cyrl': np.float64(0.46078431372549017), 'bam_Latn': np.float64(0.25), 'ban_Latn': np.float64(0.5490196078431373), 'bel_Cyrl': np.float64(0.8235294117647058), 'bem_Latn': np.float64(0.27450980392156865), 'ben_Beng': np.float64(0.803921568627451), 'bho_Deva': np.float64(0.8088235294117647), 'bjn_Arab': np.float64(0.3284313725490196), 'bjn_Latn': np.float64(0.7598039215686274), 'bod_Tibt': np.float64(0.25), 'bos_Latn': np.float64(0.8774509803921569), 'bug_Latn': np.float64(0.25), 'bul_Cyrl': np.float64(0.8774509803921569), 'cat_Latn': np.float64(0.8774509803921569), 'ceb_Latn': np.float64(0.4411764705882353), 'ces_Latn': np.float64(0.8774509803921569), 'cjk_Latn': np.float64(0.25), 'ckb_Arab': np.float64(0.25), 'crh_Latn': np.float64(0.75), 'cym_Latn': np.float64(0.5490196078431373), 'dan_Latn': np.float64(0.9019607843137255), 'deu_Latn': np.float64(0.8921568627450981), 'dik_Latn': np.float64(0.2696078431372549), 'dyu_Latn': np.float64(0.25), 'dzo_Tibt': np.float64(0.25), 'ell_Grek': np.float64(0.8529411764705882), 'eng_Latn': np.float64(0.8921568627450981), 'epo_Latn': np.float64(0.8333333333333334), 'est_Latn': np.float64(0.8235294117647058), 'eus_Latn': np.float64(0.7990196078431373), 'ewe_Latn': np.float64(0.3382352941176471), 'fao_Latn': np.float64(0.75), 'fij_Latn': np.float64(0.3137254901960784), 'fin_Latn': np.float64(0.8578431372549019), 'fon_Latn': np.float64(0.25), 'fra_Latn': np.float64(0.8676470588235294), 'fur_Latn': np.float64(0.7058823529411765), 'fuv_Latn': np.float64(0.25980392156862747), 'gaz_Latn': np.float64(0.25), 'gla_Latn': np.float64(0.4264705882352941), 'gle_Latn': np.float64(0.47058823529411764), 'glg_Latn': np.float64(0.8725490196078431), 'grn_Latn': np.float64(0.25), 'guj_Gujr': np.float64(0.8137254901960784), 'hat_Latn': np.float64(0.25), 'hau_Latn': np.float64(0.27450980392156865), 'heb_Hebr': np.float64(0.8284313725490197), 'hin_Deva': np.float64(0.8284313725490197), 'hne_Deva': np.float64(0.7598039215686274), 'hrv_Latn': np.float64(0.8872549019607843), 'hun_Latn': np.float64(0.8823529411764706), 'hye_Armn': np.float64(0.8480392156862745), 'ibo_Latn': np.float64(0.23039215686274508), 'ilo_Latn': np.float64(0.5882352941176471), 'ind_Latn': np.float64(0.8872549019607843), 'isl_Latn': np.float64(0.8284313725490197), 'ita_Latn': np.float64(0.25), 'jav_Latn': np.float64(0.6274509803921569), 'jpn_Jpan': np.float64(0.8921568627450981), 'kab_Latn': np.float64(0.22549019607843138), 'kac_Latn': np.float64(0.22549019607843138), 'kam_Latn': np.float64(0.25), 'kan_Knda': np.float64(0.8431372549019608), 'kas_Arab': np.float64(0.25), 'kas_Deva': np.float64(0.25), 'kat_Geor': np.float64(0.8137254901960784), 'kaz_Cyrl': np.float64(0.8333333333333334), 'kbp_Latn': np.float64(0.3137254901960784), 'kea_Latn': np.float64(0.25), 'khk_Cyrl': np.float64(0.7647058823529411), 'khm_Khmr': np.float64(0.8431372549019608), 'kik_Latn': np.float64(0.25), 'kin_Latn': np.float64(0.25), 'kir_Cyrl': np.float64(0.8088235294117647), 'kmb_Latn': np.float64(0.25), 'kmr_Latn': np.float64(0.25), 'knc_Arab': np.float64(0.3431372549019608), 'knc_Latn': np.float64(0.27941176470588236), 'kon_Latn': np.float64(0.25), 'kor_Hang': np.float64(0.8627450980392157), 'lao_Laoo': np.float64(0.8774509803921569), 'lij_Latn': np.float64(0.5588235294117647), 'lim_Latn': np.float64(0.25), 'lin_Latn': np.float64(0.36764705882352944), 'lit_Latn': np.float64(0.8872549019607843), 'lmo_Latn': np.float64(0.5441176470588235), 'ltg_Latn': np.float64(0.6617647058823529), 'ltz_Latn': np.float64(0.5980392156862745), 'lua_Latn': np.float64(0.3333333333333333), 'lug_Latn': np.float64(0.25), 'luo_Latn': np.float64(0.35294117647058826), 'lus_Latn': np.float64(0.39705882352941174), 'lvs_Latn': np.float64(0.8529411764705882), 'mag_Deva': np.float64(0.8235294117647058), 'mai_Deva': np.float64(0.8382352941176471), 'mal_Mlym': np.float64(0.8431372549019608), 'mar_Deva': np.float64(0.7156862745098039), 'min_Arab': np.float64(0.25), 'min_Latn': np.float64(0.7352941176470589), 'mkd_Cyrl': np.float64(0.8431372549019608), 'mlt_Latn': np.float64(0.31862745098039214), 'mni_Beng': np.float64(0.28921568627450983), 'mos_Latn': np.float64(0.25), 'mri_Latn': np.float64(0.29901960784313725), 'mya_Mymr': np.float64(0.7941176470588235), 'nld_Latn': np.float64(0.8921568627450981), 'nno_Latn': np.float64(0.8431372549019608), 'nob_Latn': np.float64(0.8529411764705882), 'npi_Deva': np.float64(0.8431372549019608), 'nqo_Nkoo': np.float64(0.25), 'nso_Latn': np.float64(0.28921568627450983), 'nus_Latn': np.float64(0.25), 'nya_Latn': np.float64(0.25), 'oci_Latn': np.float64(0.8235294117647058), 'ory_Orya': np.float64(0.4019607843137255), 'pag_Latn': np.float64(0.6127450980392157), 'pan_Guru': np.float64(0.7843137254901961), 'pap_Latn': np.float64(0.7401960784313726), 'pbt_Arab': np.float64(0.7205882352941176), 'pes_Arab': np.float64(0.8823529411764706), 'plt_Latn': np.float64(0.25), 'pol_Latn': np.float64(0.8578431372549019), 'por_Latn': np.float64(0.8970588235294118), 'prs_Arab': np.float64(0.8333333333333334), 'quy_Latn': np.float64(0.3431372549019608), 'ron_Latn': np.float64(0.8921568627450981), 'run_Latn': np.float64(0.27450980392156865), 'rus_Cyrl': np.float64(0.8676470588235294), 'sag_Latn': np.float64(0.3137254901960784), 'san_Deva': np.float64(0.2549019607843137), 'sat_Olck': np.float64(0.25), 'scn_Latn': np.float64(0.4803921568627451), 'shn_Mymr': np.float64(0.25), 'sin_Sinh': np.float64(0.8333333333333334), 'slk_Latn': np.float64(0.8627450980392157), 'slv_Latn': np.float64(0.8725490196078431), 'smo_Latn': np.float64(0.25), 'sna_Latn': np.float64(0.2549019607843137), 'snd_Arab': np.float64(0.75), 'som_Latn': np.float64(0.3382352941176471), 'sot_Latn': np.float64(0.36764705882352944), 'spa_Latn': np.float64(0.8725490196078431), 'srd_Latn': np.float64(0.6519607843137255), 'srp_Cyrl': np.float64(0.8431372549019608), 'ssw_Latn': np.float64(0.31862745098039214), 'sun_Latn': np.float64(0.6274509803921569), 'swe_Latn': np.float64(0.8872549019607843), 'swh_Latn': np.float64(0.7303921568627451), 'szl_Latn': np.float64(0.31862745098039214), 'tam_Taml': np.float64(0.8137254901960784), 'taq_Latn': np.float64(0.3382352941176471), 'taq_Tfng': np.float64(0.25), 'tat_Cyrl': np.float64(0.6813725490196079), 'tel_Telu': np.float64(0.8284313725490197), 'tgk_Cyrl': np.float64(0.23039215686274508), 'tgl_Latn': np.float64(0.8088235294117647), 'tha_Thai': np.float64(0.8774509803921569), 'tir_Ethi': np.float64(0.25), 'tpi_Latn': np.float64(0.25), 'tsn_Latn': np.float64(0.3284313725490196), 'tso_Latn': np.float64(0.27450980392156865), 'tuk_Latn': np.float64(0.25), 'tum_Latn': np.float64(0.2696078431372549), 'tur_Latn': np.float64(0.8676470588235294), 'twi_Latn': np.float64(0.24019607843137256), 'tzm_Tfng': np.float64(0.25), 'uig_Arab': np.float64(0.6911764705882353), 'ukr_Cyrl': np.float64(0.8823529411764706), 'umb_Latn': np.float64(0.25), 'urd_Arab': np.float64(0.7990196078431373), 'uzn_Latn': np.float64(0.6666666666666666), 'vec_Latn': np.float64(0.25), 'vie_Latn': np.float64(0.8970588235294118), 'war_Latn': np.float64(0.25), 'wol_Latn': np.float64(0.27941176470588236), 'xho_Latn': np.float64(0.25), 'ydd_Hebr': np.float64(0.30392156862745096), 'yor_Latn': np.float64(0.25), 'yue_Hant': np.float64(0.8970588235294118), 'zho_Hans': np.float64(0.9068627450980392), 'zho_Hant': np.float64(0.8872549019607843), 'zsm_Latn': np.float64(0.6225490196078431), 'zul_Latn': np.float64(0.25980392156862747)}

In [20]:
import pandas as pd
df = pd.read_csv('sib_gpt4o_results_march2025.csv')
code_to_name_map = df.set_index('language code')['Language name'].to_dict()
df = pd.DataFrame({
    "language_name": [code_to_name_map[language_code] for language_code in sib_languages],
    "language_code": sib_languages,
    "Accuracy": [round(result_accuracies[language_code]*100, 1) for language_code in sib_languages]
})

# Save to CSV
df.to_csv("sib_xlmr_results_prompt2.csv", index=False)